In [0]:
import pyspark.sql.types as T
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
silver = spark.table("workspace.weather.silver_openweather")

silver = silver.withColumn("data", F.to_date(F.col("data_observacao")))

ultima_data = silver.agg(F.max("data").alias("d")).first()["d"]
if ultima_data is None:
    raise ValueError("Silver sem dados para agregar.")

janela = silver.filter(F.col("data") == F.lit(ultima_data))


kpis = (janela.groupBy("id_cidade","nome_cidade","data")
               .agg(
                   F.avg("temperatura_c").alias("temp_media"),
                   F.min("temperatura_min_c").alias("temp_min"),
                   F.max("temperatura_max_c").alias("temp_max"),
                   F.avg("umidade_pct").alias("umidade_media"),
                   F.avg("velocidade_vento_ms").alias("velocidade_vento_media"),
                   F.count(F.lit(1)).alias("n_observacoes")
               )
               .select("id_cidade","nome_cidade","data",
                       "temp_media","temp_min","temp_max",
                       "umidade_media","velocidade_vento_media","n_observacoes")
)

DeltaTable.forName(spark, "workspace.weather.gold_weather_kpis_diarios") \
  .alias("t") \
  .merge(kpis.alias("s"), "t.id_cidade = s.id_cidade AND t.data = s.data") \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()

In [0]:
display(
  spark.table("workspace.weather.gold_weather_kpis_diarios")
       .orderBy(F.col("data").desc(), F.col("nome_cidade"))
       .limit(30))